In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup
import requests

import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab
usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


Scraping the wiki table

In [2]:
BASE_URL = ['https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M']
postal_code=[]
for b in BASE_URL:
    html = requests.get(b).text
    soup = BeautifulSoup(html, "html.parser")
    #identify table we want to scrape
    postal_table = soup.find('table', {"class" : "wikitable sortable"})

try:
#loop through table, grab each of the 3 columns shown 
    for row in postal_table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) == 3:
            postal_code.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))
except: pass  


Creating a dataframe with scraped data and adjusting it

In [3]:
postal_array = np.asarray(postal_code)
df = pd.DataFrame(postal_array)
df.columns = ['PostalCode', 'Borough', 'Neighborhood'] 
result1 = df[df.Borough != 'Not assigned']
result2 = result1.groupby(['PostalCode', 'Borough']).agg(', '.join).reset_index()
result3 = result2.replace({'Neighborhood': 'Not assigned'}, {'Neighborhood': result2.Borough}, regex=True)

In [4]:
result3.shape

(103, 3)

In [5]:
locations = pd.read_csv('https://cocl.us/Geospatial_data')
result3.insert(3, 'Latitude', locations.Latitude)
result3.insert(4, 'Longitude', locations.Longitude)

In [6]:
result3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [7]:
toronto_df = result3[result3.Borough.str.contains('Toronto')]
toronto_df = toronto_df.drop('PostalCode', axis=1)

In [8]:
toronto_df.shape

(38, 4)

In [9]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_df.drop(columns = ['Neighborhood','Borough'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 1, 1, 1, 1, 1, 1])

In [10]:
toronto_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_grouped_clustering.head()

,Cluster Labels,Latitude,Longitude
37,3,43.676357,-79.293031
41,3,43.679557,-79.352188
42,3,43.668999,-79.315572
43,3,43.659526,-79.340923
44,1,43.728020,-79.388790


In [11]:
# merge toronto_grouped with toronto_df to add latitude/longitude for each neighborhood
merged_df = pd.merge(toronto_grouped_clustering,toronto_df, on = ['Latitude','Longitude'], how = 'left')
merged_df.head()

,Cluster Labels,Latitude,Longitude,Borough,Neighborhood
0,3,43.676357,-79.293031,East Toronto,The Beaches
1,3,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
2,3,43.668999,-79.315572,East Toronto,"The Beaches West, India Bazaar"
3,3,43.659526,-79.340923,East Toronto,Studio District
4,1,43.728020,-79.388790,Central Toronto,Lawrence Park


In [15]:
# create map
latitude = 43.6532
longitude = -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Neighborhood'], merged_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters